# Import the relevant packages


In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

# Data

In [6]:
mnist_dataset,mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [7]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [8]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,dtype=tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,dtype=tf.int64)


def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [9]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model
Outline the model

In [10]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

# Choose the optimizer and the loss function

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training

In [13]:
NUM_EPOCHS = 10

early_stoping = tf.keras.callbacks.EarlyStopping()

model.fit(train_data, 
          epochs = NUM_EPOCHS,
          callbacks = [early_stoping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2)

Epoch 1/10
540/540 - 5s - loss: 0.4930 - accuracy: 0.8722 - val_loss: 0.2255 - val_accuracy: 0.9365
Epoch 2/10
540/540 - 3s - loss: 0.1854 - accuracy: 0.9458 - val_loss: 0.1470 - val_accuracy: 0.9583
Epoch 3/10
540/540 - 3s - loss: 0.1289 - accuracy: 0.9622 - val_loss: 0.1121 - val_accuracy: 0.9673
Epoch 4/10
540/540 - 3s - loss: 0.1014 - accuracy: 0.9704 - val_loss: 0.0917 - val_accuracy: 0.9738
Epoch 5/10
540/540 - 3s - loss: 0.0808 - accuracy: 0.9764 - val_loss: 0.0712 - val_accuracy: 0.9798
Epoch 6/10
540/540 - 3s - loss: 0.0648 - accuracy: 0.9806 - val_loss: 0.0593 - val_accuracy: 0.9822
Epoch 7/10
540/540 - 3s - loss: 0.0552 - accuracy: 0.9841 - val_loss: 0.0500 - val_accuracy: 0.9848
Epoch 8/10
540/540 - 3s - loss: 0.0461 - accuracy: 0.9863 - val_loss: 0.0419 - val_accuracy: 0.9878
Epoch 9/10
540/540 - 3s - loss: 0.0374 - accuracy: 0.9896 - val_loss: 0.0383 - val_accuracy: 0.9875
Epoch 10/10
540/540 - 3s - loss: 0.0333 - accuracy: 0.9903 - val_loss: 0.0387 - val_accuracy: 0.9893

# Test the model

In [14]:
test_data, test_accuracy = model.evaluate(test_data)
print(test_data, test_accuracy)

1/1 [==============================] - 0s 2ms/step - loss: 0.0788 - accuracy: 0.9773
0.07877910137176514 0.9772999882698059
